# 04. 解釈（Interpretation）

## このNotebookで答えること
本プロジェクトの最終モデル **Model_C** を使って、次を「文章で説明できる状態」にする。

1. **結果の意味づけ**：どの要因が、どちら向きに、どれくらい離職率と関係していたか  
2. **なぜこの構造になったか**：4変数に収束した理由（代替変数の落ち方も含む）  
3. **示唆**：現場（業務）・経営（採用/配置/投資）・政策（制度/支援）へどう落とすか  

> 注意：都道府県の集計データ（n=47）による **相関（説明）モデル**。  
> **因果**（「これを上げると離職が必ず下がる」）は主張しない。


## 0. 最終モデルの再掲（Model_C）

- 目的変数：`turnover_total`（都道府県別 看護師離職率, %）
- 説明変数：
  - `rent_private`（民営家賃）
  - `home_ownership_rate`（持ち家率）
  - `night_shift_72h_plus`（夜勤72時間超の比率）
  - `job_openings_ratio`（求人倍率）

モデル式（線形回帰）：

\[
\text{turnover\_total} = \beta_0
+ \beta_1\,\text{rent\_private}
+ \beta_2\,\text{home\_ownership\_rate}
+ \beta_3\,\text{night\_shift\_72h\_plus}
+ \beta_4\,\text{job\_openings\_ratio}
\]

このNotebookでは、**係数の符号・大きさ・仮説**を、現場の言葉に翻訳する。


In [ ]:
# ---------------------------------------------------------
# 再現用：Model_C をもう一度フィットして、係数表を出す
# （03_modeling.ipynb と同じデータ・前処理を想定）
# ---------------------------------------------------------
import pandas as pd
import statsmodels.api as sm
from pathlib import Path

DATA_PATH = Path("../data/processed/nurse_data_clean.csv")
df = pd.read_csv(DATA_PATH)

target = "turnover_total"
x_cols = ["rent_private","home_ownership_rate","night_shift_72h_plus","job_openings_ratio"]

X = sm.add_constant(df[x_cols])
y = df[target]

m = sm.OLS(y, X).fit()

# 係数表（最小限）
coef_tbl = pd.DataFrame({
    "coef": m.params,
    "std_err": m.bse,
    "p_value": m.pvalues
}).sort_values("p_value")
coef_tbl


## 1. 係数の読み方（まず“単位”を揃える）

03_modeling で得られた Model_C の推定値（実行ログ）：

| 変数 | 係数（coef） | 向き | 直感的な読み（例） |
|---|---:|---|---|
| `rent_private` | 0.000096 | ＋ | 家賃が高いほど、離職率が高い傾向 |
| `home_ownership_rate` | -0.027591 | － | 持ち家率が高いほど、離職率が低い傾向（※本モデルでは有意でない） |
| `night_shift_72h_plus` | 0.088705 | ＋ | 過重な夜勤が多いほど、離職率が高い傾向 |
| `job_openings_ratio` | -3.986475 | － | 求人倍率が高いほど、離職率が低い傾向（要解釈） |
| 定数項 `const` | 9.022335 | — | すべて0のときの基準値（現実的には解釈しない） |

### 「1単位」問題（そのまま読むとズレる）
- 家賃は「1円」増えても意味が薄い  
- 比率（%）は「1%ポイント」増が妥当  
- 求人倍率は「0.1」刻みで見ると直感に合う  

なので、以降は **“現場感のある刻み”** で読み替える。


## 2. 変数ごとの解釈（Model_C）

### 2-1. `rent_private`（家賃）＝ 生活コスト・都市性の圧力
**係数：+0.000096（%ポイント / 家賃1円）**

- 直感的には  
  **家賃が +1万円 → 離職率が +0.96%ポイント** の関係（他の条件が同じなら）
- 解釈の核：家賃は「住居費」そのものに加えて、  
  **都市部の生活コスト／通勤・保育・人材流動性／医療需要の密度**などの“都市性”をまとめて背負う代理変数になりやすい。

**仮説（現場の言葉）**
- 「同じ給料なら、家賃が高い地域ほど生活がきつい」→離職・転職の圧力  
- 「都市部は病院が多く、転職先が見つかりやすい」→流動性が上がる  
- 「コストが高い地域ほど、世帯状況（共働き/子育て/介護）と夜勤負荷が衝突しやすい」

**示唆**
- 個人の根性ではなく、**固定費（住居費）×賃金構造**の問題として説明できる


### 2-2. `night_shift_72h_plus`（夜勤過重）＝ 労働環境の圧力
**係数：+0.0887（%ポイント / 1%ポイント）**（※変数が比率[%]で入っている想定）

- 直感的には  
  **夜勤72h超の人が +10%ポイント → 離職率が +0.887%ポイント** の関係
- 解釈の核：夜勤は **体力・睡眠・家庭生活**に直撃し、燃え尽きやすい。  
  集計データでも、ここが“構造要因”として残ったのは納得感が強い。

**仮説**
- 夜勤過重は「配置・人員不足・急性期比率・運用の硬さ」の結果  
- 離職はその帰結（ただし逆方向＝離職が人員不足を悪化させ夜勤が増える、もありえる）

**示唆**
- 現場改善で触れるレバー：  
  - 夜勤回数の上限管理／勤務間インターバル  
  - 夜勤専従・短時間正規・柔軟シフト  
  - 夜勤負荷が高い部署への支援（配置転換のルール、プリセプター設計、タスクシフト）


### 2-3. `job_openings_ratio`（求人倍率）＝ 市場要因（ただし符号は“要説明”）
**係数：-3.99（%ポイント / 求人倍率1.0）**

- 直感的には  
  **求人倍率が +0.1 → 離職率が 約 -0.40%ポイント** の関係
- ここは“直感に反する”可能性があるので、**仮説を複数用意**するのが安全。

**あり得る解釈（候補）**
1. **待遇改善仮説**：求人が強い地域ほど、病院側が賃金・手当・福利厚生で引き止めに動き、結果として離職が下がる  
2. **都市性の調整後効果**：家賃（都市性）と夜勤を入れると、求人倍率の“残差部分”は  
   「医療人材の需給が比較的マシ」「採用が回っている」側を表す可能性  
3. **測定対象のズレ**：求人倍率が「全職種」など看護市場とズレている場合、符号が直感とずれることがある

**この変数は、04で “一番の見せ場”**
- 面接やGitHubでは、  
  「符号が直感と違う → だから追加検証をする」という流れが、むしろ強い。


### 2-4. `home_ownership_rate`（持ち家率）＝ 定住性・生活安定（ただし今回は非有意）
**係数：-0.0276（p値は高め）**

- 向きとしては  
  **持ち家率が高い（定住しやすい）地域ほど、離職が低い** は直感に合う
- ただし Model_C では統計的に強くない（p値が高い）  
  → 「効いていない」ではなく、次のどちらかが多い。

**なぜ非有意になりうる？**
- 家賃や都市性（`rent_private`）と **同じ情報を部分的に共有**していて、  
  “独立した寄与”が小さくなる
- 47都道府県（n=47）だと、推定の不確実性が大きくなる

**扱い方（文章のコツ）**
- 「持ち家率は “独立効果としては弱いが、方向性は整合的”」  
- 「家賃が都市性の圧力を代表し、持ち家率は補助的に残った」


## 3. なぜ「この4変数」に収束したのか

### 3-1. 役割分担で見ると腹落ちしやすい
Model_C は、次の **4つの役割**が揃った構造になっている。

- **生活環境**：`rent_private`（生活コスト・都市性の圧力）
- **生活の固定度**：`home_ownership_rate`（定住性・家計の安定の代理）
- **労働環境**：`night_shift_72h_plus`（負荷・運用・人員不足の圧力）
- **市場環境**：`job_openings_ratio`（需給・採用難易度・待遇競争）

→ 「個人の根性」ではなく、**環境のレイヤー**で説明できる形に落ちている。

### 3-2. `commute_time` が落ちた理由（あなたの結論を“筋”で説明）
`commute_time` は相関ランキングでは上位に来るが、モデルに入れると弱くなりやすい。

- 通勤時間は **都市性の副産物**になりがち  
  （都市＝家賃高い・人口密度高い・交通混雑、など）
- そのため `rent_private` を入れた時点で、通勤時間が持つ情報は **かなり吸収される**
- さらに n=47 だと、冗長な変数を足すほど  
  **係数が不安定 → 汎化（LOOCV）が落ちる** 方向に働きやすい

要するに：
> `commute_time` は「それ自体が原因」というより  
> **都市性の影**を測っている。  
> だから家賃を入れると、追加で入れる意味が薄くなる。


## 4. モデルとしての健全性（“言える範囲”を明確化）

03_modeling の診断結果（要点）：
- 調整済みR²：**0.69**（都道府県差の約7割を説明）
- LOOCV R²：**0.61**（47都道府県での外挿に近い妥当性）
- VIF：すべて概ね低く、極端な多重共線性は目立ちにくい

### 4-1. それでも「因果」は言わない理由
- 都道府県集計なので、個人レベルの意思決定とは階層が違う（生態学的誤謬）
- 夜勤や求人は、離職の結果として悪化する（逆因果）可能性
- 都市性（人口・病院数・賃金・教育機関など）の未観測要因が残り得る

### 4-2. それでも価値がある理由
- 「個人の根性」ではなく、**構造（環境）で説明できた**  
- 説明変数が少なく、**現場・経営・政策の会話に落としやすい**


## 5. 示唆（現場・経営・政策）

### 5-1. 現場（看護管理・師長）
- “夜勤過重”は離職の構造要因として残る  
  → まず **夜勤設計の見直し（上限/間隔/負荷集中部署の支援）**
- 家賃が高い地域では、業務改善だけでは限界  
  → **住宅・通勤・保育**など生活と接続した支援が必要

### 5-2. 経営（人事・採用・病院運営）
- 都市部（家賃高）ほど、賃金以外の **実質手取り**が落ちる  
  → 住宅手当、寮、借上げ、交通費、子育て支援を“離職対策投資”として整理
- 夜勤負荷は運用問題でもある  
  → 採用だけで埋めるのではなく、**配置・タスクシフト・病棟運用**の設計課題として扱う
- 求人倍率の符号は、逆に“戦略の余地”  
  → 「採用市場が厳しい地域ほど待遇改善が効く」仮説の検証価値がある

### 5-3. 政策（自治体・国）
- 都市部の離職は、医療制度だけでなく **住居費の問題**と接続  
  → 家賃補助・保育・交通の支援は、人材定着政策として合理的
- 夜勤過重は、制度と人員配置の問題  
  → 勤務間インターバル、夜勤の上限、タスクシフト推進、教育支援（新人の離脱抑制）


## 6. 次にやると、説得力が一段上がる（おすすめ順）

1. **標準化係数（β）**を出す  
   - 単位が違う4変数を、影響度で比較できる（面接で強い）
2. **影響点を除いた頑健性チェック**  
   - Cook距離上位を外しても符号が保たれるか
3. **ターゲット別（新卒/既卒）で同様に回す**  
   - “どの層が何に反応しているか”が見える
4. **交互作用**：`rent_private × night_shift_72h_plus`  
   - 「生活コストが高い地域で夜勤が重いと離職が跳ねる？」の検証
5. **求人倍率の再定義**（可能なら看護に寄せる）  
   - 変数の意味がズレている可能性を潰す

下に、標準化係数の最小コードを置く。


In [ ]:
# ---------------------------------------------------------
# 標準化係数（β）：影響度の比較用
# ---------------------------------------------------------
import numpy as np

def standardized_betas(df, target, x_cols):
    # Zスコア標準化（平均との差/標準偏差）
    z = df[[target] + x_cols].apply(lambda s: (s - s.mean())/s.std(ddof=0))
    y = z[target]
    X = sm.add_constant(z[x_cols])
    m = sm.OLS(y, X).fit()
    return m

m_std = standardized_betas(df, "turnover_total", x_cols)
pd.DataFrame({
    "beta_std": m_std.params,
    "p_value": m_std.pvalues
}).drop(index="const").sort_values("beta_std", key=np.abs, ascending=False)


---

## 7. 最後に（このNotebookの“持ち帰り”）

この分析で言えることは、シンプルに次：

- 離職は「個人の弱さ」ではなく、  
  **生活コスト（家賃）× 労働負荷（夜勤）× 市場環境（求人）**の組み合わせで説明できる
- 都市部ほど構造的に厳しくなりやすい  
  → “現場改善”と“生活支援”をセットで考える必要がある

次は、上の「おすすめ順」1〜2を回すと、04が完成形に近づく。
